<a href="https://colab.research.google.com/github/37stu37/GNS_GAN/blob/master/ctgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**import**
---



In [0]:
import pandas as pd
import pathlib as Path

In [2]:
%cd /content/drive/My Drive/Colab Notebooks/01_githubRepo/GNS_GAN

/content/drive/My Drive/Colab Notebooks/01_githubRepo/GNS_GAN


In [0]:
outputDir = '../../02_output'

In [0]:
data = pd.read_parquet(outputDir + '/smallMonteCarlo2500runs.parquet')

**Data processing**
---



In [0]:
data.reset_index(inplace=True)
data.drop(['index', 'scenario', 'pid'], axis=1, inplace=True)

In [7]:
len(data) #; data.columns

Index(['source', 'target', 'distance', 'bearing', 'IgnProb_bl'], dtype='object')

In [0]:
trainingSample = data.sample(n=70000)
discrete_columns = ['source', 'target']

In [0]:
trainingSample.info()

**CTGAN**

In [0]:
%%capture
!pip install ctgan 

In [0]:
%%time
from ctgan import CTGANSynthesizer

ctgan = CTGANSynthesizer()
ctgan.fit(trainingSample, discrete_columns)

Epoch 1, Loss G: 7.2501, Loss D: -0.0195
Epoch 2, Loss G: 6.7852, Loss D: 0.0095
Epoch 3, Loss G: 4.7395, Loss D: 0.0002
Epoch 4, Loss G: 0.5496, Loss D: -0.0637
Epoch 5, Loss G: -0.3295, Loss D: 0.0223
Epoch 6, Loss G: -0.5452, Loss D: -0.0406
Epoch 7, Loss G: -0.7048, Loss D: -0.0027
Epoch 8, Loss G: -0.8074, Loss D: 0.0254
Epoch 9, Loss G: -0.9780, Loss D: -0.0082
Epoch 10, Loss G: -1.0572, Loss D: 0.0345
Epoch 11, Loss G: -0.9820, Loss D: 0.0068
Epoch 12, Loss G: -1.0484, Loss D: -0.0465
Epoch 13, Loss G: -1.0510, Loss D: 0.0010
Epoch 14, Loss G: -1.0513, Loss D: 0.0130
Epoch 15, Loss G: -1.0337, Loss D: 0.0216
Epoch 16, Loss G: -1.1064, Loss D: 0.0597
Epoch 17, Loss G: -1.1119, Loss D: 0.0035
Epoch 18, Loss G: -1.0944, Loss D: -0.0010
Epoch 19, Loss G: -1.0848, Loss D: 0.0096
Epoch 20, Loss G: -1.0420, Loss D: 0.0295
Epoch 21, Loss G: -0.9930, Loss D: -0.0510
Epoch 22, Loss G: -0.9908, Loss D: 0.0941
Epoch 23, Loss G: -0.9213, Loss D: 0.0115
Epoch 24, Loss G: -0.8075, Loss D: -0.0

**Generate synthetic data**

In [0]:
samples = ctgan.sample(1000)

**Validation**
---



In [0]:
from FFEpostprocessing.ipynb import load_shapefile, count_fid_occurences, merge_coordinates_export_shape # comment out some def

In [0]:
shapefile = load_shapefile("buildings_raw.shp", 1748570, 5425500, 1749500, 5427600) # small

In [0]:
gdf_count_GAN = merge_coordinates_export_shape(count_df, gdf, "smallBurnedBuildings")